**Importing the dependencies**

In [1]:
import os
import json
import requests
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.authentication import AzureCliAuthentication

**Create a Resource Group from Azure Portal**

In [2]:
cli_auth = AzureCliAuthentication()
ws = Workspace.from_config(auth=cli_auth)

In [3]:
ws.get_details()

{'id': '/subscriptions/682339fd-134e-4b0f-adf5-5e65f349ff43/resourceGroups/deployy/providers/Microsoft.MachineLearningServices/workspaces/deployy',
 'name': 'deployy',
 'identity': {'principal_id': '8b9dabe3-f047-4e8b-b461-8978530347cb',
  'tenant_id': '0bc92751-071a-4e2c-a48b-633206fef374',
  'type': 'SystemAssigned'},
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '46af1613-1211-45ef-aede-1b07b4b82e41',
 'sdkTelemetryAppInsightsKey': 'e1f7b545-6243-4abf-ba76-c5691d2edb62',
 'description': '',
 'friendlyName': 'deployy',
 'creationTime': '2024-11-06T23:10:21.4274258Z',
 'containerRegistry': '/subscriptions/682339fd-134e-4b0f-adf5-5e65f349ff43/resourceGroups/deployy/providers/Microsoft.ContainerRegistry/registries/46af1613121145efaede1b07b4b82e41',
 'keyVault': '/subscriptions/682339fd-134e-4b0f-adf5-5e65f349ff43/resourceGroups/deployy/providers/Microsoft.Keyvault/vaults/deployy9788750305',
 'applicationIns

In [4]:
!pip install azure-storage-blob

In [5]:
storage_account_key = "ggn3Njd2pcTLEqkzHbGCiKryaOil37GitnAv3gvGX2Ri7KzncwNSWI0bho5jFDVzchKjNtjOCuc2+AStrWpGoA=="
storage_account_name = "deployy1014533338"

In [6]:
from azure.storage.blob import BlobServiceClient

# Create the connection string using the account name and key
connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)


In [8]:
model_name='chatbot'

In [9]:
import os  

# Get the current working directory  
current_directory = os.getcwd()  
print("Current Working Directory:", current_directory)  

# List all files in the current directory  
files = os.listdir(current_directory)  
print("Files in Current Directory:", files)

Current Working Directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/he304042816019012/code/Users/he30404281601901
Files in Current Directory: ['.amlignore', '.amlignore.amltmp', '.ipynb_aml_checkpoints', 'deployment (4).ipynb', 'deployment (4).ipynb.amltmp', 'modelpkl.pkl', 'score.py']


In [10]:
file_name = "modelpkl.pkl"  # Replace with your actual file name  
file_path = os.path.join(current_directory, file_name)  

# Check if the file exists and print the file path  
if os.path.isfile(file_path):  
    print("Full Path to Your File:", file_path)  
else:  
    print("File not found in the current directory.") 

Full Path to Your File: /mnt/batch/tasks/shared/LS_root/mounts/clusters/he304042816019012/code/Users/he30404281601901/modelpkl.pkl


In [11]:
model_path = file_path

In [26]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)
print(f"Registered model: {registered_model.name}, version: {registered_model.version}")

Registering model chatbot
Registered model: chatbot, version: 7


In [48]:
!pip install azureml-contrib-services 

In [49]:
# Create Custom Environment with Version-Specific Dependencies
env = Environment(name="custom_ml_environment")
deps = CondaDependencies()

# Specify Python version
deps.set_python_version("3.8")

# Add Conda packages with specific versions for compatibility
deps.add_conda_package("pandas==1.3.3")
deps.add_conda_package("nltk==3.6.3")
deps.add_conda_package("scikit-learn==0.24.2")
deps.add_conda_package("numpy==1.21.2")

# Add pip packages with specific versions for compatibility
deps.add_pip_package("datasets")
deps.add_pip_package("contractions")
deps.add_pip_package("num2words")
deps.add_pip_package("evaluate")
deps.add_pip_package("torch")
deps.add_pip_package("azureml-contrib-services")
deps.add_pip_package("transformers")
deps.add_pip_package("azureml-defaults")
deps.add_pip_package("azureml-core")  # Recommended for Azure ML resources
deps.add_pip_package("tokenizers")
env.python.conda_dependencies = deps

In [50]:
# Register the environment
env.register(ws)

{
    "assetId": "azureml://locations/eastus2/workspaces/46af1613-1211-45ef-aede-1b07b4b82e41/environments/custom_ml_environment/versions/3",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "custom_ml_envir

In [51]:
%%writefile ././score.py
import json
import torch
from azureml.core.model import Model
from transformers import AutoModelForCausalLM, AutoTokenizer
from azureml.contrib.services.aml_response import AMLResponse

# Initialize the model and tokenizer as global variables
model = None
tokenizer = None

def init():
    global model
    global tokenizer
    
    try:
        # Adjust the workspace parameter if needed and make sure the correct model version is used
        model_path = Model.get_model_path(model_name='chatbot',version='7')  # Omit version and workspace here for simplicity
        
        # Load the tokenizer and model using the path provided by Azure ML
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForCausalLM.from_pretrained(model_path)
        
        print("Model and tokenizer successfully loaded.")
    except Exception as e:
        print(f"Error during model initialization: {e}")

def run(raw_data):
    try:
        # Parse input data
        data = json.loads(raw_data)
        input_text = data.get("input_text", "")
        
        if not input_text:
            return AMLResponse("No input text provided", status_code=400)
        
        # Tokenize input text and generate response
        inputs = tokenizer(input_text, return_tensors="pt")
        outputs = model.generate(inputs["input_ids"], max_length=50)  # max_length adjusted for typical chatbot responses
        response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Return JSON response
        return AMLResponse(json.dumps({"response": response_text}), status_code=200)
    
    except Exception as e:
        return AMLResponse(f"Error during inference: {str(e)}", status_code=500)


Overwriting ././score.py


In [52]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=env)

In [53]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=8, auth_enabled=True)

In [55]:
service = Model.deploy(workspace=ws,
                       name='chatbot-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_3469/3632607535.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-11-08 10:38:33+00:00 Registering the environment.
2024-11-08 10:38:34+00:00 Building image..
2024-11-08 10:52:33+00:00 Generating deployment configuration..
2024-11-08 10:52:35+00:00 Submitting deployment to compute.
2024-11-08 10:52:40+00:00 Checking the status of deployment chatbot-service..
2024-11-08 10:57:22+00:00 Checking the status of inference endpoint chatbot-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [47]:
print(service.get_logs())

None


In [56]:
scoring_uri = service.scoring_uri

In [57]:
scoring_uri

'http://8877b5ca-e8cc-4c8c-89de-cd70256576f4.eastus2.azurecontainer.io/score'

**Cleaning up all the created resources**

In [54]:

service_name = "chatbot-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-11-08 10:38:18+00:00 Check and wait for operation (b1e5a633-40af-4670-b443-bbd183effe46) to finish.
2024-11-08 10:38:21+00:00 Deleting service entity.
Succeeded


In [18]:
# Specify the name of the registered model to delete
'''model_name = 'chatbot'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()'''


1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources